#EDA 수행


In [ ]:
import yaml
from src.preprocessing import DataCollector, DataPreprocessor
from src.utils.eda_utils import EDAAnalyzer

# 설정 로드
with open('config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# 데이터 수집
collector = DataCollector(config)
df = collector.load_data()
if not collector.validate_data(df):
    raise ValueError("데이터 검증 실패")

# EDA 수행
analyzer = EDAAnalyzer(config)

# 결측치 분석
missing_stats = analyzer.analyze_missing_values(df)
print("결측치 분석 결과:")
print(missing_stats)

# 분포 분석
analyzer.analyze_distributions(df, config['data']['numerical_features'])

# 상관관계 분석
analyzer.analyze_correlations(df, config['data']['numerical_features'])

# 데이터 전처리
preprocessor = DataPreprocessor(config)
df_processed = preprocessor.handle_missing_values(df)
df_processed = preprocessor.handle_outliers(df_processed)
df_processed = preprocessor.encode_categorical_features(df_processed)

# 전처리 결과 확인
print("\n전처리 전후 데이터 형태:")
print(f"전처리 전: {df.shape}")
print(f"전처리 후: {df_processed.shape}")

# Model training

In [ ]:
from src.models.xgb_model import XGBoostModel
from src.train import ModelTrainer

# 학습 실행
trainer = ModelTrainer('config/xgboost_config.yaml', XGBoostModel)
model = trainer.train_model()

# 또는 하이퍼파라미터 탐색
trainer.hyperparameter_search()

In [ ]:
# 1. 기본 로깅 설정
from src.utils.logger import Logger

# 로거 생성
logger = Logger(
    name='real_estate_ml',
    log_dir='logs',
    level=logging.INFO
).get_logger()

# 로그 기록
logger.info("데이터 로딩 시작")
logger.warning("결측치 발견")
logger.error("모델 학습 실패", exc_info=True)  # 예외 정보 포함

# 2. 모델 학습에서 사용
class ModelTrainer:
    def __init__(self, config_path: str, model_class: Type[BaseModel]):
        self.config = self._load_config(config_path)
        self.model_class = model_class
        self.logger = Logger(
            name=f"{model_class.__name__}Trainer",
            log_dir='logs/models'
        ).get_logger()
        
    def train_model(self):
        self.logger.info("모델 학습 시작")
        try:
            # WandB 로깅 설정
            Logger.setup_wandb_logging(
                project_name=self.config.get('wandb_project', 'real-estate-price'),
                run_name=f"{self.model_class.__name__}_{datetime.now():%Y%m%d_%H%M}",
                config=self.config
            )
            
            # 데이터 준비
            self.logger.info("데이터 준비 중...")
            train_data = self.prepare_data()
            
            # 모델 학습
            self.logger.info("모델 학습 중...")
            model = self.model_class(self.config)
            model.train(*train_data)
            
            self.logger.info("모델 학습 완료")
            return model
            
        except Exception as e:
            self.logger.error(f"모델 학습 실패: {str(e)}", exc_info=True)
            raise

In [ ]:
# 시각화 객체 생성
visualizer = Visualizer(save_dir='visualizations')

# 1. 학습 히스토리 시각화
history = {
    'loss': [...],
    'val_loss': [...],
    'rmse': [...],
    'val_rmse': [...]
}
visualizer.plot_training_history(
    history,
    metrics=['loss', 'rmse'],
    save_path='visualizations/training_history.png'
)

# 2. 회귀 결과 시각화
visualizer.plot_regression_results(
    y_true=y_test,
    y_pred=y_pred,
    save_path='visualizations/regression_results.png'
)

# 3. 잔차 분석
visualizer.plot_residuals(
    y_true=y_test,
    y_pred=y_pred,
    save_path='visualizations/residuals.png'
)

# 4. 특성 중요도 시각화
feature_importance = model.get_feature_importance()
visualizer.plot_feature_importance(
    importance=feature_importance,
    save_path='visualizations/feature_importance.png'
)